In [30]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [31]:
#Read data from multiple CSV
circuits = pd.read_csv('../data/circuits.csv', index_col=0, na_values=r'\N')
constructorResults = pd.read_csv('../data/constructor_results.csv', index_col=0, na_values=r'\N')
constructors = pd.read_csv('../data/constructors.csv', index_col=0, na_values=r'\N')
constructorStandings = pd.read_csv('../data/constructor_standings.csv', index_col=0, na_values=r'\N')
drivers = pd.read_csv('../data/drivers.csv', index_col=0, na_values=r'\N')
driverStandings = pd.read_csv('../data/driver_standings.csv', index_col=0, na_values=r'\N')
lapTimes = pd.read_csv('../data/lap_times.csv')
pitStops = pd.read_csv('../data/pit_stops.csv')
qualifying = pd.read_csv('../data/qualifying.csv', index_col=0, na_values=r'\N')
races = pd.read_csv('../data/races.csv', na_values=r'\N')
results = pd.read_csv('../data/results.csv', index_col=0, na_values=r'\N')
seasons = pd.read_csv('../data/seasons.csv', index_col=0, na_values=r'\N')
status = pd.read_csv('../data/status.csv', index_col=0, na_values=r'\N')

In [32]:
constructor_color_map = {
    'Toro Rosso': '#0000FF',
    'Mercedes': '#6CD3BF',
    'Red Bull': '#1E5BC6',
    'Ferrari': '#ED1C24',
    'Williams': '#37BEDD',
    'Force India': '#FF80C7',
    'Virgin': '#c82e37',
    'Renault': '#FFD800',
    'McLaren': '#F58020',
    'Sauber': '#006EFF',
    'Lotus': '#FFB800',
    'HRT': '#b2945e',
    'Caterham': '#0b361f',
    'Lotus F1': '#FFB800',
    'Marussia': '#6E0000',
    'Manor Marussia': '#6E0000',
    'Haas F1 Team': '#B6BABD',
    'Racing Point': '#F596C8',
    'Aston Martin': '#2D826D',
    'Alfa Romeo': '#B12039',
    'AlphaTauri': '#4E7C9B',
    'Alpine F1 Team': '#2293D1'
}

In [33]:
drivers_color_map = {
    'Lewis Hamilton': '#6CD3BF',
    'George Russel': '#6CD3BF',
    'Max Verstappen': '#1E5BC6',
    'Checo Perez': '#1E5BC6',
    'Charles Leclerc': '#ED1C24',
    'Carlos Sainz': '#ED1C24',
    'Lando Norris': '#F58020',
    'Daniel Ricciardo': '#F58020',
    'Fernando Alonso': '#2293D1',
    'Esteban Ocon': '#2293D1'
}

In [34]:
# Post-reading formatting
drivers = drivers.rename(columns={'nationality': 'driverNationality', 'url': 'driverUrl'})
drivers['driverName'] = drivers['forename'] + ' ' + drivers['surname']

constructors = constructors.rename(
    columns={'name': 'constructorName', 'nationality': 'constructorNationality', 'url': 'constructorUrl'})

races.set_index('raceId', inplace=True)
races['date'] = races['date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

pitStops = pitStops.rename(columns={'time': 'pitTime'})
pitStops['seconds'] = pitStops['milliseconds'].apply(lambda x: x / 1000)

results['seconds'] = results['milliseconds'].apply(lambda x: x / 1000)

circuits = circuits.rename(
    columns={'name': 'circuitName', 'location': 'circuitLocation', 'country': 'circuitCountry', 'url': 'circuitUrl'})

In [35]:
newResults = pd.merge(results, races, left_on='raceId', right_index=True, how='left')
newResults = pd.merge(newResults, constructors, left_on='constructorId', right_index=True, how='left')
newResults = pd.merge(newResults, drivers, left_on='driverId', right_index=True, how='left')
newResults = pd.merge(newResults,circuits,left_on='circuitId',right_index=True,how='left')

In [36]:
newPitStops = pd.merge(pitStops, races, left_on='raceId', right_index=True, how='left')
newPitStops = pd.merge(newPitStops, newResults[['raceId', 'driverId', 'driverName', 'constructorId', 'constructorName']], left_on=['raceId', 'driverId'], right_on=['raceId', 'driverId'])

In [37]:
df_drivers_and_constructors = newResults[['driverName', 'constructorName']]
df_drivers_and_constructors = df_drivers_and_constructors.drop_duplicates(subset='driverName')

,driverName,constructorName
resultId,,
1,Lewis Hamilton,McLaren
2,Nick Heidfeld,BMW Sauber
3,Nico Rosberg,Williams
4,Fernando Alonso,Renault
5,Heikki Kovalainen,McLaren
...,...,...
24974,Yuki Tsunoda,AlphaTauri
24981,Mick Schumacher,Haas F1 Team
24985,Nikita Mazepin,Haas F1 Team


In [68]:
#Variables which will be used many times
year = 2021
champion = "Max Verstappen"
focusedRace = 'French Grand Prix'
driver_focused = ['Max Verstappen', 'Charles Leclerc', 'Lewis Hamilton']

In [39]:
#Pit stop duration by constructor
fig = px.box(newPitStops[(newPitStops['seconds'] < 50) & (newPitStops['year'] == year)].groupby(
    by=['raceId', 'name', 'date', 'constructorName']).mean().reset_index().sort_values(by='seconds', ascending=True),
             x='constructorName',
             y='seconds',
             color='constructorName',
             color_discrete_map=constructor_color_map,
             width=1000,
             height=500
             )
fig.update_layout(
    title_text=f'Pit Stop Durations by Constructor for {year} Season',
)
fig.show()


C:\Users\Thoma\AppData\Local\Temp\ipykernel_19848\868017080.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [40]:
df_driver_focus = newResults[(newResults['driverName'].isin(driver_focused)) & (newResults['year'] == year)]
df_pos_max = newResults[(newResults['year'] == year)]
df_driver_focus["position"] = df_driver_focus["position"].fillna(df_pos_max["position"].max())

C:\Users\Thoma\AppData\Local\Temp\ipykernel_19848\557472804.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
fig = px.line(df_driver_focus,
              x='name',
              y='position',
              color='driverName',
              color_discrete_map=drivers_color_map,
              width=1000,
              height=500,
              title=f'Positions per race during the {year} season')
fig.update_yaxes(nticks=int(df_pos_max['position'].max()))
fig.update_yaxes(range=[df_pos_max['position'].max(), 1])
fig.update_xaxes(nticks=int(df_driver_focus['round'].max()))
fig.show()

In [51]:
df_focus_on_race = pd.merge(lapTimes,
                            newResults[['raceId', 'driverId', 'driverName', 'constructorId', 'constructorName', 'name', 'year']],
                            left_on=['raceId', 'driverId'],
                            right_on=['raceId', 'driverId'])
df_focus_on_race = df_focus_on_race[(df_focus_on_race['name'] == focusedRace) & (df_focus_on_race['driverName'].isin(driver_focused)) & (df_focus_on_race['year'] == year)]

In [69]:
fig = px.line(df_focus_on_race,
              x='lap',
              y='milliseconds',
              markers=True,
              color='driverName',
              color_discrete_map=drivers_color_map,
              title=f'Time per laps during the {focusedRace} race of {year}')
fig.show()

In [74]:
df_points_focus = newResults[(newResults['year'] == year)]
df_points_focus = df_points_focus[['driverName', 'constructorName', 'points']]
df_points_focus = df_points_focus.groupby("driverName").agg(constructorName = ('constructorName', 'first'), points = ('points', 'mean')).reset_index()
df_points_focus = df_points_focus.sort_values("points", ascending=True)
df_points_focus

,driverName,constructorName,points
16,Robert Kubica,Alfa Romeo,0.000000
14,Nikita Mazepin,Haas F1 Team,0.000000
12,Mick Schumacher,Haas F1 Team,0.000000
0,Antonio Giovinazzi,Alfa Romeo,0.136364
13,Nicholas Latifi,Williams,0.318182
7,Kimi Räikkönen,Alfa Romeo,0.500000
6,George Russell,Williams,0.727273
20,Yuki Tsunoda,AlphaTauri,1.454545
8,Lance Stroll,Aston Martin,1.545455
17,Sebastian Vettel,Aston Martin,1.954545


In [75]:
#Mean of points per race
fig = px.bar(df_points_focus,
             x='driverName',
             y='points',
             color='constructorName',
             color_discrete_map=constructor_color_map,
             title=f'Mean of points per race during the {year} season')
fig.show()

In [46]:
df_nb_accident = newResults[(newResults['year'] == year) & (newResults['position'].isnull())]
df_nb_accident = df_nb_accident[['driverName', 'constructorName']]

,driverName,constructorName
resultId,,
25425,Pierre Gasly,AlphaTauri
25440,Valtteri Bottas,Alfa Romeo
25441,Fernando Alonso,Alpine F1 Team
25442,Daniel Ricciardo,McLaren
25443,Nicholas Latifi,Williams
...,...,...
25805,Yuki Tsunoda,AlphaTauri
25823,Lando Norris,McLaren
25824,Kevin Magnussen,Haas F1 Team


In [47]:
grouped_counts = df_nb_accident.groupby('driverName').agg(constructorName = ('constructorName', 'first'), count = ('driverName', 'size')).reset_index()
grouped_counts = grouped_counts.sort_values('count', ascending=True)

,driverName,constructorName,count
6,George Russell,Mercedes,1
11,Lewis Hamilton,Mercedes,1
12,Max Verstappen,Red Bull,1
9,Lance Stroll,Aston Martin,2
17,Sergio Pérez,Red Bull,2
4,Esteban Ocon,Alpine F1 Team,2
16,Sebastian Vettel,Aston Martin,2
10,Lando Norris,McLaren,2
2,Charles Leclerc,Ferrari,3
3,Daniel Ricciardo,McLaren,3


In [76]:
# Number of unfinished race per driver
fig = px.bar(grouped_counts,
             x='driverName',
             y='count',
             color='constructorName',
             color_discrete_map=constructor_color_map,
             title=f'Number of unfinished races per driver during the {year} season')
fig.show()

In [58]:
df_focus_on_quali = pd.merge(newResults, qualifying, left_on='raceId', right_index=True, how='left')
df_focus_on_quali = df_focus_on_quali[(df_focus_on_quali['year'] == year)]
df_focus_on_quali = df_focus_on_quali[['driverName','q1', 'q2', 'q3']]
print(df_focus_on_quali)

               driverName        q1        q2        q3
resultId                                               
25406     Charles Leclerc  1:44.378  1:44.336  1:44.992
25407        Carlos Sainz  1:44.378  1:44.336  1:44.992
25408      Lewis Hamilton  1:44.378  1:44.336  1:44.992
25409      George Russell  1:44.378  1:44.336  1:44.992
25410     Kevin Magnussen  1:44.378  1:44.336  1:44.992
...                   ...       ...       ...       ...
25841     Mick Schumacher  1:31.279  1:28.954  1:29.711
25842     Kevin Magnussen  1:31.279  1:28.954  1:29.711
25843      Lewis Hamilton  1:31.279  1:28.954  1:29.711
25844     Nicholas Latifi  1:31.279  1:28.954  1:29.711
25845     Fernando Alonso  1:31.279  1:28.954  1:29.711

[440 rows x 4 columns]


In [56]:
nb_q1 = df_focus_on_quali.groupby('driverName').agg(Q1_count = ('q1', 'size'))
print(nb_q1)

                  Q1_count
driverName                
Alexander Albon         21
Carlos Sainz            22
Charles Leclerc         22
Daniel Ricciardo        22
Esteban Ocon            22
Fernando Alonso         22
George Russell          22
Guanyu Zhou             22
Kevin Magnussen         22
Lance Stroll            22
Lando Norris            22
Lewis Hamilton          22
Max Verstappen          22
Mick Schumacher         22
Nicholas Latifi         22
Nico Hülkenberg          2
Nyck de Vries            1
Pierre Gasly            22
Sebastian Vettel        20
Sergio Pérez            22
Valtteri Bottas         22
Yuki Tsunoda            22


In [60]:
nb_q2 = df_focus_on_quali.dropna(subset=['q2']).groupby('driverName').agg(Q2_count = ('q2', 'size'))
print(nb_q2)

                  Q2_count
driverName                
Alexander Albon         17
Carlos Sainz            17
Charles Leclerc         17
Daniel Ricciardo        17
Esteban Ocon            17
Fernando Alonso         17
George Russell          17
Guanyu Zhou             17
Kevin Magnussen         17
Lance Stroll            17
Lando Norris            17
Lewis Hamilton          17
Max Verstappen          17
Mick Schumacher         17
Nicholas Latifi         17
Nico Hülkenberg          2
Pierre Gasly            17
Sebastian Vettel        15
Sergio Pérez            17
Valtteri Bottas         17
Yuki Tsunoda            17
